# PAL case study global analysis

This case study concerns the fluorescence measured from the PAL mutant of a cyanobacterium, which contains only photosystem I (PSI) and photosystem II (PSII). With PSI the difference PAL_opentween open and closed reaction centers (RC) is negligible in the fluorescence, because both the oxidized and reduced RC efficiently quench the excitations. However, with PSII there is a huge difference, with closed RCs exhibiting a huge amount of recombination fluorescence. The measurement and the global analysis are described in ([Tian et al. 2013][tian2013]).
The target analysis is based upon the model explained in ([van Stokkum 2018][vanstokkum2018]).

[tian2013]: https://doi.org/10.1039/C3CP43813A "Tian L, Farooq S, van Amerongen H (2013) Probing the picosecond kinetics of the photosystem II core complex in vivo. Physical Chemistry Chemical Physics 15 (9):3146-3154."
[vanstokkum2018]: https://www.few.vu.nl/~ivo/pub/2018/2018GTAofLHandPQinvivo.pdf "Systems biophysics: Global and target analysis of light harvesting and photochemical quenching in vivo. In: Croce R, van Grondelle R, van Amerongen H, van Stokkum I (eds) Light Harvesting in Photosynthesis. CRC Press, Boca Raton, pp 467-482."

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

PAL_open_data_path = "PAL_700_ma_tr2_gain50_10uW_590nmfilter_21C_400nm_AV_bc_sh_sel_620_830.ascii"
plot_data_overview(PAL_open_data_path, nr_of_data_svd_vectors=3, title="PAL open");

## Global Analysis

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

PAL_open_global_model_path = "models/PAL_open-global-model.yml"
PAL_open_global_parameters_path = "models/PAL_open-global-parameters.yml"

#### Model file

In [ ]:
display_file(PAL_open_global_model_path, syntax="yaml")

#### Parameters file

In [ ]:
display_file(PAL_open_global_parameters_path, syntax="yaml")

The free parameters are the four kinetic parameters and the IRF parameters. The IRF consists of a sum of two Gaussians with a common center. The IRF parameters are the center (the location of the maximum of the IRF) and the widths of the two Gaussians, and the relative amplitude of the second Gaussian.
Note that two dispersion parameters (to describe the wavelength dependence of the IRF center with a parabola) have been used, which have been estimated from the data as well.

### Create scheme and optimize it

In [ ]:
from glotaran.project.scheme import Scheme

PAL_open_global_scheme = Scheme(
    model=PAL_open_global_model_path,
    parameters=PAL_open_global_parameters_path,
    data={"PAL_open_data": PAL_open_data_path},
    maximum_number_function_evaluations=11,
)
PAL_open_global_scheme.validate()

In [ ]:
compartments = PAL_open_global_scheme.model.initial_concentration["input1"].compartments

PAL_open_global_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
from glotaran.optimization.optimize import optimize

PAL_open_global_result = optimize(PAL_open_global_scheme)

In [ ]:
PAL_open_global_result

In [ ]:
PAL_open_global_result.optimized_parameters

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(
    PAL_open_global_result.data["PAL_open_data"], wavelength_range=(670, 750)
)
plot_fitted_traces(PAL_open_global_result, wavelengths, linlog=True, linthresh=50);

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    PAL_open_global_result,
    linlog=False,
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
);

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    PAL_open_global_result,
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=50,
);